# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna


from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Loading

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [5]:
def create_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']
    # blueHerald列の負の値を0で置き換え
    df['blueHerald'] = df['blueHerald'].clip(lower=0)

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # ゴールドと経験値の相互作用
    df['goldXexperience'] = df['blueTotalGold'] * df['blueTotalExperience']

    # 初期キルの影響
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    
    # キル、デス、アシストの合計
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    # キル・デス・アシスト比率
    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    # 効率的なリソース利用
    df['goldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['experiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1

    # エリートモンスターの討伐効率
    df['eliteMonstersPerGold'] = df['blueEliteMonsters'] / df['blueTotalGold']
    df['eliteMonstersPerExperience'] = df['blueEliteMonsters'] / df['blueTotalExperience']

    # チームの総合力
    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']

    return df

- Apply data

In [6]:
train = create_features(train)
test = create_features(test)

# model

- LightGBM ensemble

In [7]:
# データの準備
X = train.drop('blueWins', axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

def objective(trial, X_train_fold, y_train_fold, X_val_fold, y_val_fold):
    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }

    lgb_train = lgb.Dataset(X_train_fold, y_train_fold)
    lgb_val = lgb.Dataset(X_val_fold, y_val_fold, reference=lgb_train)

    early_stopping = lgb.early_stopping(stopping_rounds=10)
    model = lgb.train(param, lgb_train, num_boost_round=1000, valid_sets=[lgb_val], callbacks=[early_stopping])

    y_pred = model.predict(X_val_fold, num_iteration=model.best_iteration)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_val_fold, y_pred)

    return accuracy

def train_and_predict_with_optuna(X_train, y_train, X_test, n_splits=5):
    predictions = np.zeros(len(X_test))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, X_train_fold, y_train_fold, X_val_fold, y_val_fold), n_trials=100)

        best_params = study.best_params
        best_params['objective'] = 'binary'
        best_params['metric'] = 'binary_error'
        best_params['verbosity'] = -1

        lgb_train = lgb.Dataset(X_train_fold, y_train_fold)
        model = lgb.train(best_params, lgb_train, num_boost_round=study.best_trial.number)

        predictions += model.predict(X_test, num_iteration=model.best_iteration) / n_splits
    
    return (predictions > 0.5).astype(int)

# アンサンブル学習による予測
y_test_pred_ensemble = train_and_predict_with_optuna(X, y, test)

# 予測結果の表示
print(y_test_pred_ensemble)

[I 2024-01-18 10:39:19,439] A new study created in memory with name: no-name-4892e430-f6f2-4fe1-a975-807fa287b63f


Training until validation scores don't improve for 10 rounds


[I 2024-01-18 10:39:19,908] Trial 0 finished with value: 0.80125 and parameters: {'lambda_l1': 3.596268891592326e-08, 'lambda_l2': 4.381936415107103e-07, 'num_leaves': 212, 'feature_fraction': 0.8852619892867073, 'bagging_fraction': 0.6035258740978366, 'bagging_freq': 5, 'min_child_samples': 57}. Best is trial 0 with value: 0.80125.
[I 2024-01-18 10:39:20,083] Trial 1 finished with value: 0.8025 and parameters: {'lambda_l1': 0.011681521469291387, 'lambda_l2': 1.317808717028151e-07, 'num_leaves': 154, 'feature_fraction': 0.7788883097979246, 'bagging_fraction': 0.9655404574652269, 'bagging_freq': 3, 'min_child_samples': 46}. Best is trial 1 with value: 0.8025.


Early stopping, best iteration is:
[24]	valid_0's binary_error: 0.19875
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's binary_error: 0.1975


[I 2024-01-18 10:39:20,198] Trial 2 finished with value: 0.8 and parameters: {'lambda_l1': 0.08874929339329661, 'lambda_l2': 2.77486118310611e-05, 'num_leaves': 228, 'feature_fraction': 0.8064820943243312, 'bagging_fraction': 0.5213179023640181, 'bagging_freq': 6, 'min_child_samples': 27}. Best is trial 1 with value: 0.8025.
[I 2024-01-18 10:39:20,286] Trial 3 finished with value: 0.798125 and parameters: {'lambda_l1': 0.0009441495548301185, 'lambda_l2': 2.4621004505398014e-08, 'num_leaves': 253, 'feature_fraction': 0.7530540067963869, 'bagging_fraction': 0.5495263814129833, 'bagging_freq': 2, 'min_child_samples': 45}. Best is trial 1 with value: 0.8025.


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's binary_error: 0.22625
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's binary_error: 0.219375


[I 2024-01-18 10:41:24,106] Trial 98 finished with value: 0.780625 and parameters: {'lambda_l1': 0.002944398351856965, 'lambda_l2': 1.9707066970224282e-07, 'num_leaves': 92, 'feature_fraction': 0.5152574517482118, 'bagging_fraction': 0.7837964580539393, 'bagging_freq': 1, 'min_child_samples': 17}. Best is trial 2 with value: 0.789375.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	valid_0's binary_error: 0.219375
Training until validation scores don't improve for 10 rounds


[I 2024-01-18 10:41:24,470] Trial 99 finished with value: 0.77875 and parameters: {'lambda_l1': 0.01437226741758982, 'lambda_l2': 1.2813420749582267e-08, 'num_leaves': 75, 'feature_fraction': 0.4484488637609031, 'bagging_fraction': 0.7658677702233918, 'bagging_freq': 7, 'min_child_samples': 13}. Best is trial 2 with value: 0.789375.


Early stopping, best iteration is:
[37]	valid_0's binary_error: 0.22125
[1 1 0 ... 1 1 0]


- testにy_test_pred結合

In [8]:
output = '../data/output/submit_cv_oputuna_ensemble.csv'

In [9]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': y_test_pred_ensemble
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [10]:
check = pd.read_csv(output)

In [11]:
check

,9,1
0,15,1
1,18,0
2,23,0
3,31,0
4,32,1
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
